In [1]:
import os
os.chdir('../')
%pwd

'/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction'

In [2]:
import sys
import numpy as np
import keras
from pathlib import Path
import pickle
from timeseriesPredictor.exception import CustomException

2023-08-18 16:04:33.893354: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 16:04:33.944184: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 16:04:33.945160: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 16:04:35.025293: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def get_base_model(path:Path):
    model = keras.models.load_model(path)
    return model  


def load_pickle_file(path: Path):
    file= open(path, 'rb')
    obj = pickle.load(file)
    return obj


autoencoder = get_base_model('artifacts/training_autoencoder/tensor_model.h5')

data = load_pickle_file('artifacts/data_transformation/tensor/train.pkl')
data.shape

(635, 100, 64, 1)

In [4]:
list(autoencoder.input_shape)

[None, 100, 64, 1]

In [5]:
encoderLayer = autoencoder.get_layer("encoder")
assert(encoderLayer != None)

decoderLayer = autoencoder.get_layer("decoder")
assert(decoderLayer != None)

encoder = keras.Model(
autoencoder.input, 
encoderLayer.output
)

decoder = keras.Model(
encoderLayer.output, 
decoderLayer.output
)

In [7]:
encoder.predict(data)

20/20 [==============================] - 1s 22ms/step


array([[[[-8.47831666e-02, -2.28291109e-01,  8.54863882e-01],
         [-8.47831666e-02, -2.43416116e-01,  8.54863882e-01],
         [ 1.88516691e-01,  6.94761217e-01,  8.54863882e-01],
         ...,
         [ 7.42317200e-01,  4.41134900e-01,  8.19952250e-01],
         [ 3.47694531e-02,  8.40316653e-01,  8.76635790e-01],
         [ 7.95442641e-01,  7.56135941e-01,  4.22065586e-01]],

        [[-1.78081080e-01, -2.43416116e-01,  8.54863882e-01],
         [-1.78081080e-01, -2.43416116e-01,  8.54863882e-01],
         [ 1.59699127e-01, -6.48967475e-02,  8.54863882e-01],
         ...,
         [ 4.59524989e-01,  2.28910685e-01,  1.06143427e+00],
         [ 1.82189673e-01, -8.90405476e-02,  9.93560791e-01],
         [ 9.55419540e-01, -1.27519116e-01,  1.00639486e+00]],

        [[-1.78081080e-01, -2.43416116e-01,  8.54863882e-01],
         [-1.78081080e-01, -2.43416116e-01,  8.54863882e-01],
         [-1.78081080e-01, -2.43416116e-01,  8.54863882e-01],
         ...,
         [ 1.15645587e+0

In [6]:
x = data[0:1]
x_decoder = encoder.predict(x)
decoder.predict(x_decoder).shape

1/1 [==============================] - 0s 109ms/step


(1, 100, 64, 1)

In [7]:
x = data
encoded = encoder.predict(x)
encoded.shape

20/20 [==============================] - 0s 18ms/step


(635, 25, 16, 3)

In [57]:
def videos_frame(encoded, lag):    
    videos = np.empty((encoded.shape[0] -lag, encoded.shape[1], encoded.shape[2], encoded.shape[3], lag))
    next_frame = np.empty((encoded.shape[0] -lag, encoded.shape[1], encoded.shape[2], encoded.shape[3], 1))

    for i in range(len(encoded)):    
        for j in range(lag):  
            if (i) < len(videos):     
                videos[i, :, :, :, j] = encoded[j+i]            
                next_frame[i,:,:,:,0] = encoded[i+j+1]            
            else:
                break
    return videos, next_frame

(628, 25, 16, 3, 7)
0 0
0 1
0 2
0 3
0 4
0 5
0 6
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 2
2 3
2 4
2 5
2 6
2 7
2 8
3 3
3 4
3 5
3 6
3 7
3 8
3 9
4 4
4 5
4 6
4 7
4 8
4 9
4 10
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 6
6 7
6 8
6 9
6 10
6 11
6 12
7 7
7 8
7 9
7 10
7 11
7 12
7 13
8 8
8 9
8 10
8 11
8 12
8 13
8 14
9 9
9 10
9 11
9 12
9 13
9 14
9 15
10 10
10 11
10 12
10 13
10 14
10 15
10 16
11 11
11 12
11 13
11 14
11 15
11 16
11 17
12 12
12 13
12 14
12 15
12 16
12 17
12 18
13 13
13 14
13 15
13 16
13 17
13 18
13 19
14 14
14 15
14 16
14 17
14 18
14 19
14 20
15 15
15 16
15 17
15 18
15 19
15 20
15 21
16 16
16 17
16 18
16 19
16 20
16 21
16 22
17 17
17 18
17 19
17 20
17 21
17 22
17 23
18 18
18 19
18 20
18 21
18 22
18 23
18 24
19 19
19 20
19 21
19 22
19 23
19 24
19 25
20 20
20 21
20 22
20 23
20 24
20 25
20 26
21 21
21 22
21 23
21 24
21 25
21 26
21 27
22 22
22 23
22 24
22 25
22 26
22 27
22 28
23 23
23 24
23 25
23 26
23 27
23 28
23 29
24 24
24 25
24 26
24 27
24 28
24 29
24 30
25 25
25 26
25 27
25 28
25 29
25 30
25 31
26 26
2

In [35]:
videos[0][:,:,:,2]

array([[[-0.08478317, -0.22829111,  0.85486388],
        [-0.08478317, -0.24341612,  0.85486388],
        [ 0.1919491 ,  0.23584831,  0.85486388],
        ...,
        [ 0.16376086,  0.02871837,  0.94087011],
        [-0.01574749, -0.24341612,  0.89726043],
        [ 0.1429189 ,  0.28556257,  0.8213259 ]],

       [[-0.17808108, -0.24341612,  0.85486388],
        [-0.17808108, -0.24341612,  0.85486388],
        [-0.06391589, -0.24341612,  0.85486388],
        ...,
        [ 0.02430795, -0.20833933,  1.00603759],
        [-0.14966539, -0.24341612,  0.87728965],
        [ 0.10010067, -0.1397604 ,  0.85486388]],

       [[-0.17808108, -0.24341612,  0.85486388],
        [-0.17808108, -0.24341612,  0.85486388],
        [-0.17808108, -0.24341612,  0.85486388],
        ...,
        [ 0.42594975,  0.44544894,  0.87572718],
        [ 0.08989687,  0.10830542,  0.92748487],
        [ 0.03581377,  0.55412775,  0.87481374]],

       ...,

       [[-0.17808108, -0.24341612,  0.85486388],
        [-0

In [6]:
def split_sequence(sequence, lag):
        '''
        This function splits a given univariate sequence into
        multiple samples where each sample has a specified number
        of time steps and the output is a single time step.
        param new_input: If True it is used for predicting new input
        '''
        try:
                
            X, y = list(), list()
            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix = i + lag
                # check if we are beyond the sequence               
                if end_ix > len(sequence)-1:
                    break
            # gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return np.array(X), np.array(y)
    
        except Exception as e:
            raise CustomException(e, sys)
    
def convert_to_supervised(dat, lag):
        '''
        This function takes a 2D sequennce, scales the array and splits
        a given multivariate sequence into multiple samples where each sample has a specified number
        of time steps. It returns multiple time steps as well as the scaler.
        param df (DataFrame): Bike sharing demand for each community over time
        param lag (int): History length or time lag
        '''
        
        try:
            if isinstance(dat, np.ndarray):
                pass
            else:
                dat = dat.values
            
            m, n = dat.shape
            # e.g., if lag = 7, BIXI demand of past 7*15 minutes
            X = np.zeros((m-lag,lag, n))
            Y = np.zeros((m-lag,n))

            for i in range(0,n):
                x, y = split_sequence(dat[:,i], lag)
                X[:,:,i] = x
                Y[:,i] = y
            return X, Y
    
        except Exception as e:
            raise CustomException(e, sys)   

In [7]:
lag = 7
X, y = convert_to_supervised(dat, lag)
train_x = X.reshape(X.shape[0], 7, 100,64, 1)
train_y = y.reshape(y.shape[0], 100, 64, 1)
X.shape

(628, 7, 6400)

In [8]:
y.shape

(628, 6400)

In [9]:
print(train_x.shape)
print(train_y.shape)

(628, 7, 100, 64, 1)
(628, 100, 64, 1)


In [21]:
# define encoder
input = keras.Input(shape= (7, 100, 64, 1))
encoder = keras.layers.ConvLSTM2D(filters=64, kernel_size=(3,3), activation='relu', return_sequences=True)(input)
encoder = keras.layers.BatchNormalization()(encoder)
encoder = keras.layers.ConvLSTM2D(filters=32, kernel_size=(3,3), activation='relu', return_sequences=True)(encoder)
encoder = keras.layers.BatchNormalization()(encoder)
encoder = keras.layers.Flatten()(encoder)
# define predict decoder
decoder= keras.layers.RepeatVector(100*64)(encoder)
decoder= keras.layers.LSTM(64, activation='relu', return_sequences=True)(decoder)
decoder = keras.layers.BatchNormalization()(decoder)
decoder= keras.layers.LSTM(32, activation='relu', return_sequences=True)(decoder)
decoder = keras.layers.BatchNormalization()(decoder)
decoder = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='relu'))(decoder)
decoder = keras.layers.Reshape(target_shape=(100, 64, 1))(decoder)
decoder = keras.layers.Convolution2D(filters=1, kernel_size=(3, 3), activation='linear', padding='same')(decoder) 


model = keras.Model(inputs= input, outputs=decoder)
model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss = 'mean_squared_error' ,
            metrics=['MSE']           
                           )

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 7, 100, 64, 1)]   0         
                                                                 
 conv_lstm2d_22 (ConvLSTM2D  (None, 7, 98, 62, 64)     150016    
 )                                                               
                                                                 
 batch_normalization_33 (Ba  (None, 7, 98, 62, 64)     256       
 tchNormalization)                                               
                                                                 
 conv_lstm2d_23 (ConvLSTM2D  (None, 7, 96, 60, 32)     110720    
 )                                                               
                                                                 
 batch_normalization_34 (Ba  (None, 7, 96, 60, 32)     128       
 tchNormalization)                                         

In [10]:
history = model.fit(train_x, train_y, epochs=1, batch_size=1, verbose=1)      

2023-08-16 10:16:41.919380: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1269760000 exceeds 10% of free system memory.
2023-08-16 10:16:42.085583: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1269760000 exceeds 10% of free system memory.


In [23]:
# define encoder
input = keras.Input(shape= (7, 100, 64, 1))
encoder = keras.layers.ConvLSTM2D(filters=64, kernel_size=(3,3), activation='relu', return_sequences=True)(input)
encoder = keras.layers.BatchNormalization()(encoder)
encoder = encoded = keras.layers.ConvLSTM2D(filters=18, kernel_size=(3, 3), activation='relu', padding='same')(encoded)(keras.layers.MaxPooling2D(5,5))(encoder)
encoder = keras.layers.ConvLSTM2D(filters=32, kernel_size=(3,3), activation='relu', return_sequences=True)(encoder)
encoder = keras.layers.BatchNormalization()(encoder)
encoder = keras.layers.TimeDistributed(keras.layers.MaxPooling2D(5,5))(encoder)
encoder = keras.layers.Flatten()(encoder)
encoder = keras.layers.RepeatVector(3)(encoder)


model = keras.Model(inputs= input, outputs=encoder)
model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss = 'mean_squared_error' ,
            metrics=['MSE']           
                           )

model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 7, 100, 64, 1)]   0         
                                                                 
 conv_lstm2d_26 (ConvLSTM2D  (None, 7, 98, 62, 64)     150016    
 )                                                               
                                                                 
 batch_normalization_39 (Ba  (None, 7, 98, 62, 64)     256       
 tchNormalization)                                               
                                                                 
 time_distributed_16 (TimeD  (None, 7, 19, 12, 64)     0         
 istributed)                                                     
                                                                 
 conv_lstm2d_27 (ConvLSTM2D  (None, 7, 17, 10, 32)     110720    
 )                                                         